<a href="https://colab.research.google.com/github/zahra-zarrabi/FaceRecognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

configs = {
              "learning_rate": 0.001,
              "epochs": 5,
              "batch_size": 32
           }

wandb.login()
run = wandb.init(project='face-recognition', config=configs)
config = wandb.config


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **👩‍🍳 Prepare Dataset**

In [16]:
image_data_generator = ImageDataGenerator(rescale=1./255,
                                          horizontal_flip = True,
                                          validation_split= 0.2)

train_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=True,
                                                      target_size=(224, 224),
                                                      subset='training')

val_data = image_data_generator.flow_from_directory('/content/drive/MyDrive/7-7 dataset',
                                                      class_mode='categorical',
                                                      batch_size=config.batch_size,
                                                      shuffle=False,
                                                      target_size=(224, 224),
                                                      subset='validation')

Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize = (8,8))
for i in range(16):
  plt.subplot(4,4,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  # plt.imshow(train_images[i])
  print(train_images.shape)

# 🧠 Define the Model and the Training Loop

In [17]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.base=tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')
    # self.conv1=Conv2D(32,(3,3),activation='relu')
    # self.conv2=Conv2D(16,(3,3),activation='relu')
    self.flatten=Flatten()
    self.fc1=Dense(256,activation='relu')
    self.fc2=Dense(128,activation='relu')
    self.fc3=Dense(number_of_classes, activation='softmax')
    
  def call(self,x):    
    for laye in self.base.layers[:-8]:
      laye.trainable=False
    x=self.base(x)
    # x = self.conv1(x)
    # x = self.conv2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    output = self.fc3(x)
    return output

In [ ]:
model.base.summary()

In [ ]:
model.summary()

In [20]:
model = MyModel(14)

In [9]:
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_acc = tf.keras.metrics.CategoricalAccuracy(name='val_acc')


In [11]:
def train_step(images,y):
  with tf.GradientTape() as tape:
    y_pred = model(images,training=True)
    # print('y_pred',y_pred)
    # print('y', y.shape)
    loss = loss_function(y,y_pred)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  # print('y', y)
  # print('y_pred', y_pred)
  train_acc(y,y_pred)
  # print('jnjnj')
 

In [12]:
def val_step(images,y):
  y_pred = model(images,training=False)
  loss = loss_function(y,y_pred)

  val_loss(loss)
  val_acc(y,y_pred)

In [22]:
def train():
  epochs=config.epochs
  for epoch in range(epochs):

    train_loss.reset_states()
    train_acc.reset_states()
    val_loss.reset_states()
    val_acc.reset_states()

    cnt = 0
    for cnt in tqdm(range(train_data.__len__())):
      images,labels = next(train_data)
      train_step(images,labels)
    
    for cnt in tqdm(range(val_data.__len__())):
      images,labels = next(val_data)
      val_step(images,labels)

    print('epoch:',epoch+1)
    print('loss:',train_loss.result())
    print('accuracy:',train_acc.result())
    print('val loss:',val_loss.result())
    print('val accuracy:',val_acc.result())
    print('-------------------------------------------------------------------')


    #log metrics using wandb.log
    wandb.log({'epochs': epoch,
                'loss': train_loss.result(),
                'acc': train_acc.result(), 
                'val_loss': val_loss.result(),
                'val_acc':val_acc.result()})

In [ ]:
#fit
train()

In [ ]:
model.save_weights('/content/drive/MyDrive/model2/')

In [ ]:
model.load_weights('/content/drive/MyDrive/model1/')